In [1]:
import json
import pandas as pd
import numpy as np

In [11]:
df = pd.read_excel(
    r"word_list\japanese\NLT1.40_freq_list.xlsx",
    sheet_name="NLT 1.40頻度リスト",
)

# extract and rename columns
df = df[["読み", "頻度"]].rename(columns={"読み": "word", "頻度": "count"})

# cleanse words
df = df[np.isfinite(df["count"])]
df = df.dropna()

# fix format
df["word"] = df["word"].astype(str)
df["count"] = df["count"].astype(int)
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[df["word"].str.isalpha()]

# frequent words
df = df[df["count"] > df["count"].quantile(0.5)]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,ℓ,564,1
1,々,73050,1
2,〆,1170,1
3,っ,133403,1
4,ゝ,8109,1


In [9]:
df.dtypes

word        object
count        int32
word_len     int64
dtype: object

In [12]:
df.describe()

,count,word_len
count,3.714400e+04,37144.000000
mean,2.556308e+04,4.421979
std,5.607082e+05,1.347253
min,1.580000e+02,1.000000
25%,4.800000e+02,4.000000
50%,1.291000e+03,4.000000
75%,4.833250e+03,5.000000
max,5.726447e+07,13.000000


In [5]:
df.query("count > 10 and word_len == 3").describe()

,count,word_len
count,7.386000e+03,7386.0
mean,2.009574e+04,3.0
std,1.023878e+05,0.0
min,1.580000e+02,3.0
25%,6.790000e+02,3.0
50%,2.097000e+03,3.0
75%,8.597500e+03,3.0
max,4.111651e+06,3.0


In [14]:
df.query("'タビタビ' in word")

,word,count,word_len
19539,タビタビ,11536,4


In [7]:
difficulties = {"easy": 0.95, "medium": 0.9, "hard": 0.7, "all": 0}
lengths = [4]

for l in lengths:
    for name, percentile in difficulties.items():
        cutoff = df["count"].quantile(percentile)
        df_filtered = (
            df.query(f"count > {cutoff} and word_len == {l}")
            .sort_values(by=["count"], ascending=False)
            .reset_index(drop=True)
        )
        with open(rf"vocabs\ja\ja-len{l}-{name}.json", "w") as f:
            data = df_filtered["word"].sort_values().tolist()
            json.dump(data, f)
            print(f"{f.name}: {len(df_filtered)} words")

vocabs\ja\ja-len4-easy.json: 607 words
vocabs\ja\ja-len4-medium.json: 1242 words
vocabs\ja\ja-len4-hard.json: 3979 words
vocabs\ja\ja-len4-all.json: 13155 words
